<a href="https://colab.research.google.com/github/Hanna07111/stock-prediction-project/blob/main/lstm_sentiment_industry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/deep.daiv/Finbert-lstm/FinBERT-LSTM

/content/drive/MyDrive/Colab Notebooks/deep.daiv/Finbert-lstm/FinBERT-LSTM


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# hyperparameters
split = 0.85
sequence_length = 10
epochs = 100
learning_rate = 0.02

#기존 reuter 데이터에 산업 레이블 추가된 데이터 사용
# Load the data
stock_data = pd.read_csv("nasdaq_100_close_prices.csv")
news_data = pd.read_csv("reuters_sentiment_with_industry.csv")

# Convert the Time column to datetime in news data and Date column in stock data
news_data['Time'] = pd.to_datetime(news_data['Time'])
stock_data['Date'] = stock_data['Date'].str.split(' ').str[0]
stock_data['Date'] = pd.to_datetime(stock_data['Date']).dt.date

# Calculate returns and fill NaN values
stock_data['Close'] = (stock_data['Close'] - stock_data['Close'].shift(1)) / stock_data['Close'].shift(1)
stock_data['Close_10day'] = (stock_data['Close'] - stock_data['Close'].shift(10)) / stock_data['Close'].shift(10)
stock_data['Close'] = stock_data['Close'].fillna(0)
stock_data['Close_10day'] = stock_data['Close_10day'].fillna(0)

#산업 레이블을 숫자로 인코딩
# Encode the industry labels
label_encoder = LabelEncoder()
news_data['industry'] = label_encoder.fit_transform(news_data['industry'])

# Prepare the datasets
X = []
y = []

# Label the data
stock_data['upDown'] = stock_data['Close_10day'].apply(lambda x: 1 if x > 0 else 0)

# Iterate over each news entry
for i, row in news_data.iterrows():
    news_date = row['Time'].date()
    sentiment = row['Sentiment']
    industry = row['industry']

    # Find the index of the news date in the stock data
    idx = stock_data.index[stock_data['Date'] == news_date].tolist()
    if not idx:
        continue
    idx = idx[0]

    # Get the next 10 days of stock prices including the news date
    if idx + 10 >= len(stock_data):
        continue

    price_data = stock_data['Close'].iloc[idx:idx + 10].values

    if len(price_data) == 10:
        X.append(np.append(price_data, industry))
        y.append(stock_data['Close_10day'].iloc[idx + 10])

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)
y = np.where(y > 0, 1, 0)

# Split the data into training and testing sets
train_size = int(len(X) * split)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape data for LSTM
X_train = X_train.reshape((X_train.shape[0], sequence_length + 1, 1))
X_test = X_test.reshape((X_test.shape[0], sequence_length + 1, 1))

In [ ]:
# Creating LSTM model
def model_create():
    tf.random.set_seed(1234)
    model = tf.keras.models.Sequential(
        [
            tf.keras.Input(shape=(X_train.shape[1], 1)),
            tf.keras.layers.LSTM(units=70, activation="tanh", return_sequences=True),
            tf.keras.layers.LSTM(units=30, activation="tanh", return_sequences=True),
            tf.keras.layers.LSTM(units=10, activation="tanh", return_sequences=False),
            tf.keras.layers.Dense(units=1, activation="sigmoid")
        ]
    )

    model.compile(
        loss=tf.keras.losses.binary_crossentropy,
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['accuracy']
    )

    model.fit(
        X_train, y_train,
        epochs=epochs
    )
    return model

# Prediction on test set
def predict(model):
    predictions = model.predict(X_test)
    return predictions

# Evaluation
def evaluate(predictions):
    predictions = np.where(predictions > 0.5, 1, 0)
    mae = mean_absolute_error(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)
    accuracy = np.mean(predictions == y_test)
    return mae, mape, accuracy

# Run model
def run_model(n):
    total_mae = total_mape = total_acc = 0
    for i in range(n):
        model = model_create()
        predictions = predict(model)
        mae, mape, acc = evaluate(predictions)
        total_mae += mae
        total_mape += mape
        total_acc += acc
    return (total_mae / n), (total_mape / n), (total_acc / n)

mae, mape, acc = run_model(1)

print(f"Mean Absolute Error = {mae}")
print(f"Mean Absolute Percentage Error = {mape}%")
print(f"Accuracy = {acc}")

Epoch 1/100
783/783 [==============================] - 21s 19ms/step - loss: 0.5646 - accuracy: 0.7495
Epoch 2/100
783/783 [==============================] - 14s 18ms/step - loss: 0.5640 - accuracy: 0.7496
Epoch 3/100
783/783 [==============================] - 15s 19ms/step - loss: 0.5634 - accuracy: 0.7496
Epoch 4/100
783/783 [==============================] - 15s 19ms/step - loss: 0.5635 - accuracy: 0.7496
Epoch 5/100
783/783 [==============================] - 14s 18ms/step - loss: 0.5634 - accuracy: 0.7496
Epoch 6/100
783/783 [==============================] - 15s 19ms/step - loss: 0.5633 - accuracy: 0.7496
Epoch 7/100
783/783 [==============================] - 15s 19ms/step - loss: 0.5633 - accuracy: 0.7496
Epoch 8/100
783/783 [==============================] - 15s 19ms/step - loss: 0.5632 - accuracy: 0.7496
Epoch 9/100
783/783 [==============================] - 14s 18ms/step - loss: 0.5632 - accuracy: 0.7496
Epoch 10/100
783/783 [==============================] - 14s 18ms/step - l